In [5]:
# ===============================
# ✅ Robust Colab → GitHub uploader
# Copies: /MyDrive/Colab Notebooks/HW4  →  https://github.com/Mastermind0309/NTHU_ML_HW.git
# ===============================

import os, sys, subprocess, shutil

# ---- 1) CONFIG ----
REPO_URL   = "https://github.com/Mastermind0309/NTHU_ML_HW.git"
GIT_NAME   = "Mastermind0309"
GIT_EMAIL  = "codyhuang90@gmail.com"   # TODO: replace with your GitHub email
SRC_DIR    = "/content/drive/MyDrive/Colab Notebooks/HW4"
REPO_DIR   = "/content/NTHU_ML_HW"
COMMIT_MSG = "Upload HW4 folder from Google Drive (Colab)"

# ---- 2) Reset safe working dir BEFORE any deletes ----
try:
    os.chdir("/content")
except Exception as e:
    print("Could not chdir to /content:", e)

# ---- 3) Mount Google Drive ----
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

# Validate source directory
if not os.path.isdir(SRC_DIR):
    raise FileNotFoundError(f"Source folder not found: {SRC_DIR}")

# ---- 4) Configure git identity (per session) ----
subprocess.run(["git", "config", "--global", "user.name", GIT_NAME], check=True)
subprocess.run(["git", "config", "--global", "user.email", GIT_EMAIL], check=True)

# ---- 5) Fresh clone (or fix if clone fails) ----
# Clean any leftover broken repo folder
if os.path.exists(REPO_DIR):
    try:
        shutil.rmtree(REPO_DIR)
        print(f"Removed existing: {REPO_DIR}")
    except Exception as e:
        print("Warning: couldn't remove existing repo dir:", e)

print("Cloning repo...")
clone = subprocess.run(["git", "clone", REPO_URL, REPO_DIR])
if clone.returncode != 0:
    print("\n⚠️ Clone failed. Falling back to initializing a new repo folder locally.")
    os.makedirs(REPO_DIR, exist_ok=True)
    os.chdir(REPO_DIR)
    subprocess.run(["git", "init"], check=True)
    subprocess.run(["git", "branch", "-M", "main"], check=True)
    # set or add origin safely
    have_origin = subprocess.run(["git", "remote", "get-url", "origin"]).returncode == 0
    if have_origin:
        subprocess.run(["git", "remote", "set-url", "origin", REPO_URL], check=True)
    else:
        subprocess.run(["git", "remote", "add", "origin", REPO_URL], check=True)
else:
    os.chdir(REPO_DIR)

# ---- 6) Copy the entire HW4 folder into the repo root ----
print("Copying files from Drive → repo...")
# Prefer rsync (handles spaces; keeps repo tidy)
rsync_cmd = [
    "rsync","-av","--delete",
    "--exclude","*.ipynb_checkpoints",
    "--exclude","__pycache__",
    SRC_DIR.rstrip("/") + "/",  # ensure trailing slash
    REPO_DIR.rstrip("/") + "/"
]
rs = subprocess.run(rsync_cmd)
if rs.returncode != 0:
    print("rsync failed; falling back to shutil.copytree (merge)...")
    # fallback: copy everything (merge-like)
    for root, dirs, files in os.walk(SRC_DIR):
        rel = os.path.relpath(root, SRC_DIR)
        dst_root = os.path.join(REPO_DIR, rel) if rel != "." else REPO_DIR
        os.makedirs(dst_root, exist_ok=True)
        for d in dirs:
            os.makedirs(os.path.join(dst_root, d), exist_ok=True)
        for f in files:
            if f.endswith(".ipynb_checkpoints") or f == "__pycache__":
                continue
            shutil.copy2(os.path.join(root, f), os.path.join(dst_root, f))

# ---- 7) Commit & push ----
print("\nStaging, committing, and pushing...")
# Ensure we're in repo dir
os.chdir(REPO_DIR)

# Make sure it's a git repo (handles the fallback path)
inside_repo = subprocess.run(["git","rev-parse","--is-inside-work-tree"]).returncode == 0
if not inside_repo:
    subprocess.run(["git","init"], check=True)
    subprocess.run(["git","branch","-M","main"], check=True)
    have_origin = subprocess.run(["git","remote","get-url","origin"]).returncode == 0
    if have_origin:
        subprocess.run(["git","remote","set-url","origin", REPO_URL], check=True)
    else:
        subprocess.run(["git","remote","add","origin", REPO_URL], check=True)

subprocess.run(["git","add","."], check=True)
# Show status (for visibility in output)
subprocess.run(["git","status"])

# Commit (no error if nothing to commit)
commit = subprocess.run(["git","commit","-m", COMMIT_MSG])
if commit.returncode != 0:
    print("Nothing to commit (repo already up-to-date).")

# Ensure branch is main
subprocess.run(["git","branch","-M","main"], check=True)

# Push (will prompt for username + PAT on first push)
push = subprocess.run(["git","push","-u","origin","main"])
if push.returncode != 0:
    print("\n❗ Push failed.")
    print("   If this is your first time pushing from Colab, when prompted:")
    print("   - Username: Mastermind0309")
    print("   - Password: your GitHub Personal Access Token (PAT) with 'repo' scope.")
    print("   Generate at: GitHub → Settings → Developer settings → Personal access tokens → Tokens (classic)")
else:
    print("\n✅ Done! Check your repo: https://github.com/Mastermind0309/NTHU_ML_HW")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Removed existing: /content/NTHU_ML_HW
Cloning repo...
Copying files from Drive → repo...

Staging, committing, and pushing...

❗ Push failed.
   If this is your first time pushing from Colab, when prompted:
   - Username: Mastermind0309
   - Password: your GitHub Personal Access Token (PAT) with 'repo' scope.
   Generate at: GitHub → Settings → Developer settings → Personal access tokens → Tokens (classic)
